In [1]:
!pip install -q \
    numpy \
    pandas \
    scikit-learn \
    chardet \
    statsmodels


'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
import pandas as pd
import numpy as np
import os
import pickle

current_dir = os.getcwd()

pv_month_dir = os.path.join(current_dir, "Data", "태양광발전 월보")
pv_day_dir = os.path.join(current_dir, "Data", "태양광발전일보")
ld_dir = os.path.join(current_dir,"Data","전력일보")
fc_dir = os.path.join(current_dir,"Data","예보")

# 태양광 발전량

In [3]:
pv_day_data = {}
dlist = os.listdir(pv_day_dir)
dlist = [f for f in dlist if f != '.DS_Store']

def increase_value(index, value):
    increment = pd.to_timedelta(index, unit="H")
    return value + increment

def extract_numbers(input_string):
    return ''.join([char for char in input_string if char.isdigit()])

for dname in dlist:
    data = pd.read_excel(os.path.join(pv_day_dir, dname))

    start_date = pd.to_datetime(data.iloc[1][~data.iloc[1].isna()].values[0] , format="%Y년 %m월 %d일")

    class_index = data.iloc[2][~data.iloc[2].isna()].index.tolist()

    data_dict = {}

    for name in data.columns:
        if name in class_index:
            cname = data.loc[2, name]
            data_dict[cname] = pd.DataFrame({f"{cname}_{data.loc[3,name]}" : data[name]})
        else:
            data_dict[cname][f"{cname}_{data.loc[3,name]}"] = data[name]

    data_dict['시간']['시간'] = data_dict['시간'].pop(f"시간_{np.nan}")
    

    for key in data_dict.keys():
        if key == '시간':
            data_dict[key] = data_dict[key].iloc[5:-1].apply(extract_numbers, axis=1).reset_index(drop=True)
            data_dict[key] = pd.DataFrame({key: [increase_value(int(idx), start_date) for idx in range(len(data_dict[key]))]})
            
        else:
            try:
                data_dict[key] = data_dict[key].iloc[5:-1].reset_index(drop=True).astype(float)
            except Exception:
                data_dict[key] = data_dict[key].iloc[5:-1].reset_index(drop=True)

    pv_day_data[extract_numbers(dname)] = data_dict

with open (os.path.join(current_dir, "Data", 'pv_day_data.pkl'), 'wb') as f:
    pickle.dump(pv_day_data,f)

pv_day_merged_dict = {}
for outer_key in pv_day_data.keys():
    for inner_key in pv_day_data[outer_key].keys():
        if inner_key not in pv_day_merged_dict:
            pv_day_merged_dict[inner_key] = []
        pv_day_merged_dict[inner_key].append(pv_day_data[outer_key][inner_key])

for key in pv_day_merged_dict.keys():
    pv_day_merged_dict[key] = pd.concat(pv_day_merged_dict[key], axis=0, ignore_index=True)

with open (os.path.join(current_dir, "Data", 'pv_day_merged.pkl'), 'wb') as f:
    pickle.dump(pv_day_merged_dict,f)

merge_df = pd.DataFrame()
for key in pv_day_merged_dict.keys():
    _df = pv_day_merged_dict[key]
    merge_df = pd.concat([merge_df, _df], axis=1)

merge_df = merge_df.drop(columns=['일일 사용량_nan'])
merge_df.replace('-', np.nan, inplace=True)
merge_df = merge_df.sort_values(by='시간')
merge_df.set_index('시간', inplace=True)


# 결측치 대체, 단순하게
from sklearn.ensemble import RandomForestRegressor

columns_with_nan = merge_df.columns[merge_df.isna().any()].tolist()

for column in columns_with_nan:
    df_notna = merge_df[merge_df[column].notna()]
    df_na = merge_df[merge_df[column].isna()]
    
    X_train = df_notna.drop(columns=columns_with_nan)
    y_train = df_notna[column]
    
    X_test = df_na.drop(columns=columns_with_nan)
    
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    
    merge_df.loc[merge_df[column].isna(), column] = model.predict(X_test)


merge_df.to_csv(os.path.join(current_dir, "Data", 'pv_day_merged.csv'))

C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\2820067046.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  increment = pd.to_timedelta(index, unit="H")
C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\2820067046.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  increment = pd.to_timedelta(index, unit="H")
C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\2820067046.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  increment = pd.to_timedelta(index, unit="H")
C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\2820067046.py:6: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  increment = pd.to_timedelta(index, unit="H")
C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\2820067046.py:6: FutureWarning: 'H' is deprecated and will be removed in a

In [4]:
# pv_month_data = {}
# dlist = os.listdir(pv_month_dir)

# def extract_numbers(input_string):
#     return ''.join([char for char in input_string if char.isdigit()])

# for dname in dlist:
#     data = pd.read_excel(os.path.join(pv_month_dir, dname))

#     class_index = data.iloc[2][~data.iloc[2].isna()].index.tolist()

#     data_dict = {}

#     for name in data.columns:
#         if name in class_index:
#             cname = data.loc[2, name]
#             data_dict[cname] = pd.DataFrame({f"{cname}_{data.loc[3,name]}" : data[name]})
#         else:
#             data_dict[cname][f"{cname}_{data.loc[3,name]}"] = data[name]

#     data_dict['날짜']['날짜'] = data_dict['날짜'].pop(f"날짜_{np.nan}")

#     for key in data_dict.keys():
#         if key == '날짜':
#             data_dict[key] = data_dict[key].iloc[5:-1].apply(extract_numbers, axis=1).reset_index(drop=True)
#         else:
#             try:
#                 data_dict[key] = data_dict[key].iloc[5:-1].reset_index(drop=True).astype(float)
#             except Exception:
#                 data_dict[key] = data_dict[key].iloc[5:-1].reset_index(drop=True)

#     pv_month_data[extract_numbers(dname)] = data_dict

# with open (os.path.join(current_dir, "Data", 'pv_month_data.pkl'), 'wb') as f:
#     pickle.dump(pv_month_data,f)

# pv_month_merged_dict = {}
# for outer_key in pv_month_data.keys():
#     for inner_key in pv_month_data[outer_key].keys():
#         if inner_key not in pv_month_merged_dict:
#             pv_month_merged_dict[inner_key] = []
#         pv_month_merged_dict[inner_key].append(pv_month_data[outer_key][inner_key])

# for key in pv_month_merged_dict.keys():
#     pv_month_merged_dict[key] = pd.concat(pv_month_merged_dict[key], axis=0, ignore_index=True)

# pv_month_merged_dict['날짜'] = pd.DataFrame({'날짜': [day for day in range(1, len(pv_month_merged_dict['날짜']) + 1)]})

# with open (os.path.join(current_dir, "Data", 'pv_month_merged.pkl'), 'wb') as f:
#     pickle.dump(pv_month_merged_dict,f)

# merge_df = pd.DataFrame()
# for key in pv_month_merged_dict.keys():
#     _df = pv_month_merged_dict[key]
#     merge_df = pd.concat([merge_df, _df], axis=1)

# merge_df.replace('-', np.nan, inplace=True)


# # 결측치 대체, 단순하게
# from sklearn.ensemble import RandomForestRegressor

# columns_with_nan = merge_df.columns[merge_df.isna().any()].tolist()

# for column in columns_with_nan:
#     df_notna = merge_df[merge_df[column].notna()]
#     df_na = merge_df[merge_df[column].isna()]
    
#     X_train = df_notna.drop(columns=columns_with_nan)
#     y_train = df_notna[column]
    
#     X_test = df_na.drop(columns=columns_with_nan)
    
#     model = RandomForestRegressor(n_estimators=100, random_state=0)
#     model.fit(X_train, y_train)
    
#     merge_df.loc[merge_df[column].isna(), column] = model.predict(X_test)


# merge_df.to_csv(os.path.join(current_dir, "Data", 'pv_month_merged.csv'))

# 기상 예보

In [5]:
import re
import unicodedata

dlist = os.listdir(fc_dir)

fc_merged_df = pd.DataFrame()

def increase_value(index, value):
    increment = pd.to_timedelta(index // 508, unit="D")
    return value + increment

for filename in dlist:
    try:
        data  = pd.read_csv(os.path.join(fc_dir, filename), encoding="utf-8-sig")

        pattern = re.compile(r'_([^_]+)_(\d{8})_(\d{8})\.csv$')
        match = pattern.search(filename)

        extracted_text = match.group(1)
        start_date = match.group(2)

        start_date = pd.to_datetime(start_date, format='%Y%m%d')

        data = data.dropna(ignore_index=True)

        data.columns = ['day', 'hour', 'forecast', 'value']

        data['type'] = unicodedata.normalize('NFC',extracted_text)
        
        data['day'] = [increase_value(idx, start_date) for idx, val in enumerate(data['day'])]

        fc_merged_df = pd.concat([fc_merged_df, data], ignore_index= True)

    except Exception as e:
        print(e)
        print(filename)
        break

fc_merged_df = fc_merged_df.pivot_table(index=['day', 'hour', 'forecast'], columns='type', values=['value'], aggfunc='first')

fc_merged_df.columns = [col[1] for col in fc_merged_df.columns.values]
fc_merged_df.reset_index(inplace=True)

fc_merged_df['datetime'] = fc_merged_df['day'] + pd.to_timedelta(fc_merged_df['hour'] / 100 + fc_merged_df['forecast'], unit="H")

fc_merged_df = fc_merged_df[~(fc_merged_df['datetime'] < "2022-07-01")]
fc_merged_df = fc_merged_df[~(("2022-09-01" <= fc_merged_df['datetime']) & (fc_merged_df['datetime'] < "2023-07-01"))]
fc_merged_df = fc_merged_df[~(fc_merged_df['datetime'] >= "2023-09-01")]


# 가중합
def weighted_average(df, weight_col, value_col):
    weights = df[weight_col]
    values = df[value_col]
    return (weights * values).sum() / weights.sum()

group_by_col = 'datetime'
weight_col = 'hour'
ex_col = ['day', 'forecast']
exclude_cols = [group_by_col, weight_col] + ex_col
value_cols = [col for col in fc_merged_df.columns if col not in exclude_cols]

def calculate_weighted_averages(group):
    result = {weight_col: group[weight_col].max()}
    for col in value_cols:
        result[col] = weighted_average(group, weight_col, col)
    return pd.Series(result)

# 그룹화하여 가중합 계산
weighted_fc = fc_merged_df.groupby(group_by_col).apply(calculate_weighted_averages).reset_index()
weighted_fc.set_index('datetime', inplace=True)
weighted_fc = weighted_fc.drop(columns=['hour'])

weighted_fc.to_csv(os.path.join("Data",'forecast.csv'))
weighted_fc

C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\1930517249.py:44: FutureWarning: 'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.
  fc_merged_df['datetime'] = fc_merged_df['day'] + pd.to_timedelta(fc_merged_df['hour'] / 100 + fc_merged_df['forecast'], unit="H")
C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\1930517249.py:70: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_fc = fc_merged_df.groupby(group_by_col).apply(calculate_weighted_averages).reset_index()


,1시간강수량,1시간기온,강수형태,강수확률,남북바람성분,동서바람성분,습도,풍속,풍향,하늘상태
datetime,,,,,,,,,,
2022-07-01 00:00:00,0.0,27.200000,0.0,28.000000,2.620000,-1.080000,78.000000,2.840000,157.800000,3.800000
2022-07-01 01:00:00,0.0,28.200000,0.0,28.000000,2.680000,-1.240000,73.000000,2.920000,155.200000,3.800000
2022-07-01 02:00:00,0.0,29.259259,0.0,20.000000,2.848148,-1.340741,68.333333,3.155556,155.037037,3.000000
2022-07-01 03:00:00,0.0,30.259259,0.0,28.148148,2.985185,-1.274074,63.703704,3.274074,157.074074,3.814815
2022-07-01 04:00:00,0.0,31.259259,0.0,23.703704,2.992593,-1.166667,60.555556,3.218519,159.148148,3.370370
...,...,...,...,...,...,...,...,...,...,...
2023-08-31 19:00:00,0.0,21.913242,0.0,26.849315,-1.240183,-0.382648,92.716895,1.308676,18.045662,3.684932
2023-08-31 20:00:00,0.0,21.643777,0.0,26.437768,-1.414163,-0.385837,95.000000,1.457082,15.935622,3.643777
2023-08-31 21:00:00,0.0,21.287554,0.0,26.437768,-1.321459,-0.542918,93.927039,1.421459,22.287554,3.643777


In [6]:
# join with forecast
# pv_fc_merged = pd.concat([merge_df, weighted_fc], axis=1, join='inner')
# pv_fc_merged.to_csv(os.path.join("Data", "pv_fc_merged.csv"))
# pv_fc_merged

# 부하 전처리

In [9]:
import os
import numpy as np
import chardet
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA


file_path_s = os.path.join("Data","전력일보","2022","석사 일보.gcf_2022-07-01_23-59.xls")

with open(file_path_s, 'rb') as f:
    print(chardet.detect(f.read()))

df_s = pd.read_excel(file_path_s)

file_path_h = os.path.join("Data","전력일보","2022","학사 일보.gcf_2022-07-01_23-59.xls")

with open(file_path_h, 'rb') as f:
    print(chardet.detect(f.read()))

df_h = pd.read_excel(file_path_h)


df_s = pd.read_excel(file_path_s, header=6)
col_s = df_s.iloc[0, 1:].dropna().tolist()

df_h = pd.read_excel(file_path_h, header=6)
col_h = df_h.iloc[0, 1:].dropna().tolist()


folder_path = os.path.join("Data", "전력일보","2022")

columns_s = ['유효전력'] + [f'유효전력.{i}' for i in range(1, 27)]
columns_h = ['유효전력'] + [f'유효전력.{i}' for i in range(1, 29)]

all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

data_dict_seoksa = {col: [] for col in columns_s}
data_dict_haksa = {col: [] for col in columns_h}



for files in all_files:
    try:
        df = pd.read_excel(files, header=8)
        
        if '석사' in files:
            for col in columns_s:
                if col in df.columns:
                    data_dict_seoksa[col].extend(df.iloc[1:][col].dropna().tolist())
        elif '학사' in files:
            for col in columns_h:
                if col in df.columns:
                    data_dict_haksa[col].extend(df.iloc[1:][col].dropna().tolist())
    except Exception as e:
        print(f"Error processing file {files}: {e}")



combined_df_s = pd.DataFrame({col: data_dict_seoksa[col] for col in columns_s})
combined_df_h = pd.DataFrame({col: data_dict_haksa[col] for col in columns_h})

combined_df_s.columns = col_s
combined_df_h.columns = col_h

date_range = pd.date_range(start='2022-07-01 00:00:00', end='2022-08-31 23:00:00', freq='H')

date_range

combined_df_s['datetime'] = date_range
combined_df_h['datetime'] = date_range

combined_df_s.set_index('datetime', inplace=True)
combined_df_h.set_index('datetime', inplace=True)

buildings_s = ['전기전자컴퓨터공학동','신소재공학동','생명과학동','기계공학동','LG도서관','창업B동','금호관','기혼자아파트E동','기숙사9동','고등광/극초단']
buildings_h = ['신재생에너지동','대학B동','대학기숙사A동','제2학생회관','교원아파트','대학C동','중앙연구기기센터','대학A동','다산빌딩','산학협력연구동']

combined_df_s = combined_df_s[buildings_s]
combined_df_h = combined_df_h[buildings_h]

combined_df_2022 = combined_df_s.join(combined_df_h, how='inner')

for column in combined_df_2022.columns:
    if combined_df_2022[column].dtype == 'object':
        combined_df_2022[column] = combined_df_2022[column].astype(float)

combined_df_2022.to_csv(os.path.join("Data", '2022_load.csv'))






file_path_s = os.path.join("Data", "전력일보","2023", "석사 일보.gcf_2023-07-01_23-59.xls")

with open(file_path_s, 'rb') as f:
    print(chardet.detect(f.read()))

df_s = pd.read_excel(file_path_s)
df_s

file_path_h = os.path.join("Data", "전력일보","2023", "학사 일보.gcf_2023-07-01_23-59.xls")

with open(file_path_h, 'rb') as f:
    print(chardet.detect(f.read()))

df_h = pd.read_excel(file_path_h)
df_h


df_s = pd.read_excel(file_path_s, header=6)
col_s = df_s.iloc[0, 1:].dropna().tolist()


df_h = pd.read_excel(file_path_h, header=6)
col_h = df_h.iloc[0, 1:].dropna().tolist()


folder_path = os.path.join("Data", "전력일보","2023")

columns_s = ['유효전력'] + [f'유효전력.{i}' for i in range(1, 28)]
columns_h = ['유효전력'] + [f'유효전력.{i}' for i in range(1, 30)]

all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

data_dict_seoksa = {col: [] for col in columns_s}
data_dict_haksa = {col: [] for col in columns_h}


for files in all_files:
    try:
        df = pd.read_excel(files, header=8)
        
        if '석사' in files:
            for col in columns_s:
                if col in df.columns:
                    data_dict_seoksa[col].extend(df.iloc[1:][col].dropna().tolist())
        elif '학사' in files:
            for col in columns_h:
                if col in df.columns:
                    data_dict_haksa[col].extend(df.iloc[1:][col].dropna().tolist())
    except Exception as e:
        print(f"Error processing file {files}: {e}")



combined_df_s = pd.DataFrame({col: data_dict_seoksa[col] for col in columns_s})
combined_df_h = pd.DataFrame({col: data_dict_haksa[col] for col in columns_h})

combined_df_s.columns = col_s
combined_df_h.columns = col_h

date_range = pd.date_range(start='2023-07-01 00:00:00', end='2023-08-30 23:00:00', freq='H')



combined_df_s['datetime'] = date_range
combined_df_h['datetime'] = date_range

combined_df_s.set_index('datetime', inplace=True)
combined_df_h.set_index('datetime', inplace=True)

buildings_s = ['전기전자컴퓨터공학동','신소재공학동','생명과학동','기계공학동','LG도서관','창업B동','금호관','기혼자아파트E동','기숙사9동','고등광/극초단']
buildings_h = ['신재생에너지동','대학B동','대학기숙사A동','제2학생회관','교원아파트','대학C동','중앙연구기기센터','대학A동','다산빌딩','산학협력연구동']

combined_df_s = combined_df_s[buildings_s]
combined_df_h = combined_df_h[buildings_h]

combined_df_2023 = combined_df_s.join(combined_df_h, how='inner')


combined_df_2023.replace('-', np.nan, inplace=True)

for column in combined_df_2023.columns:
    if combined_df_2023[column].dtype == 'object':
        combined_df_2023[column] = combined_df_2023[column].astype(float)


for buildings in combined_df_2023.columns:

    missing_dates = combined_df_2023[combined_df_2023[buildings].isna()].index

    model = ARIMA(combined_df_2022[buildings], order=(1, 1, 1))
    model_fit = model.fit()

    for date in missing_dates:
        predicted_value = model_fit.forecast(steps=1)[0]
        
        combined_df_2023.at[date, buildings] = predicted_value

combined_df_2023.to_csv(os.path.join("Data", '2023_load.csv'))

{'encoding': 'IBM866', 'confidence': 0.48643436329567696, 'language': 'Russian'}
{'encoding': 'IBM866', 'confidence': 0.5647504734582799, 'language': 'Russian'}


C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\1450080424.py:65: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start='2022-07-01 00:00:00', end='2022-08-31 23:00:00', freq='H')


{'encoding': 'IBM866', 'confidence': 0.4707078324528374, 'language': 'Russian'}
{'encoding': 'IBM866', 'confidence': 0.5557389152912715, 'language': 'Russian'}


C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\1450080424.py:153: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start='2023-07-01 00:00:00', end='2023-08-30 23:00:00', freq='H')
c:\Users\jiwon\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
c:\Users\jiwon\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
c:\Users\jiwon\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency h will be used.
  self._init_dates(dates, freq)
C:\Users\jiwon\AppData\Local\Temp

# 연도별 전처리 파일 결합

In [10]:
import pandas as pd
import os 

weather1 = pd.read_csv(os.path.join("Data", "weather2022.csv"), index_col=0)
weather2 = pd.read_csv(os.path.join("Data", "weather2023.csv"), index_col=0, encoding="cp949")
weather2 = weather2[weather1.columns]
weather2.index  = pd.date_range(start="2023-07-01", end="2023-08-31 23:00:00", freq='H')
weather = pd.concat([weather1, weather2], axis=0)
weather.to_csv(os.path.join("Data", "weather.csv"))


load1    = pd.read_csv(os.path.join("Data", "2022_load.csv"), index_col=0)
load2   = pd.read_csv(os.path.join("Data", "2023_load.csv"), index_col=0)


load = pd.concat([load1, load2], axis=0)

load.to_csv(os.path.join("Data", "load.csv"))


smp1 = pd.read_csv(os.path.join("Data", "smp_land_2022.csv"), encoding="cp949")
smp2 = pd.read_csv(os.path.join("Data", "smp_land_2023.csv"), encoding="cp949")
smp = pd.concat([smp1, smp2], axis=0)
smp.index = smp['구분']
smp = smp.drop(columns=['구분'])

smp.to_csv(os.path.join("Data", "smp.csv"))

C:\Users\jiwon\AppData\Local\Temp\ipykernel_26612\1920105099.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  weather2.index  = pd.date_range(start="2023-07-01", end="2023-08-31 23:00:00", freq='H')
